In [1]:
from autotsc import utils
import os
from aeon.datasets.tsc_datasets import univariate
import random
from itertools import product
from tqdm import tqdm
import polars as pl
import numpy as np
from time import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from aeon.classification.convolution_based import RocketClassifier, MiniRocketClassifier
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from aeon.transformations.collection import Normalizer
from aeon.classification.sklearn import SklearnClassifierWrapper
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifierCV
from time import perf_counter

In [2]:
write_dir = "experiments/automl_ca_vs_time_correlation"
os.makedirs(write_dir, exist_ok=True)

datasets = list(univariate)
random.shuffle(datasets)
model_types = ['raw-scale-ridge', 'quant', 'minirocket', 'catch22', 'hydra', 'tabpfn', 'shapelet']

n_jobs = 4
n_runs = 5

In [3]:
def get_model(model_name):
    from sklearn.linear_model import RidgeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import HistGradientBoostingClassifier
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from aeon.classification.interval_based import QUANTClassifier
    from aeon.classification.feature_based import Catch22Classifier
    from aeon.classification.convolution_based import HydraClassifier
    from aeon.classification.shapelet_based import ShapeletTransformClassifier
    from tabpfn import TabPFNClassifier

    if model_name == 'raw-scale-ridge':
        return SklearnClassifierWrapper(
            make_pipeline(
                StandardScaler(),
                RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
            )
        )
    elif model_name == 'tabpfn':
        return SklearnClassifierWrapper(
            TabPFNClassifier(n_preprocessing_jobs=n_jobs)
        )
    elif model_name == 'quant':
        return QUANTClassifier()
    elif model_name == 'minirocket':
        return MiniRocketClassifier(n_jobs=n_jobs)
    elif model_name == 'catch22':
        return Catch22Classifier(n_jobs=n_jobs)
    elif model_name == 'hydra':
        return HydraClassifier(n_jobs=n_jobs)
    elif model_name == 'shapelet':
        return ShapeletTransformClassifier(n_jobs=n_jobs)
    else:
        raise ValueError(f"Unknown model: {model_name}")
        

In [4]:
all_combinations = list(product(datasets, range(n_runs), model_types))

last_dataset = None
X_train, y_train, X_test, y_test = None, None, None, None

for ds, run, model_name in tqdm(all_combinations, desc="Processing"):
    try: 
        model = get_model(model_name)
        stats = {
            "dataset": ds,
            "run": run,
            "model": model_name,
        }

        hash_val = pl.DataFrame([stats]).hash_rows(
            seed=42, seed_1=1, seed_2=2, seed_3=3
        ).item()
        file = f"{write_dir}/{hash_val}.parquet"

        if os.path.exists(file):
            #print(f'Skipping {stats}')
            continue
        else:
            print(f'Processing {stats}')

        if ds != last_dataset:
            X_train, y_train, X_test, y_test = utils.load_dataset(ds)
            last_dataset = ds

        start_time = perf_counter()
        model.fit(X_train, y_train)
        training_time = perf_counter() - start_time
        y_pred = model.predict(X_test)
        test_accuracy = accuracy_score(y_test, y_pred)

        stats["test_accuracy"] = test_accuracy
        stats["training_time"] = training_time

        df_stat = pl.DataFrame([stats])
        df_stat.write_parquet(file)

    except Exception as e:
        print(f"Error processing {ds}, run {run}, model {model_name}: {e}")
        continue

Processing:   0%|          | 1/4480 [00:00<27:27,  2.72it/s]

Processing {'dataset': 'Chinatown', 'run': 0, 'model': 'shapelet'}


Processing:   0%|          | 7/4480 [00:13<2:32:15,  2.04s/it]

Processing {'dataset': 'Chinatown', 'run': 1, 'model': 'shapelet'}


Processing:   0%|          | 14/4480 [00:21<1:50:04,  1.48s/it]

...

Processing {'dataset': 'HouseTwenty', 'run': 3, 'model': 'shapelet'}


Processing:   8%|▊         | 343/4480 [3:15:58<31:06:15, 27.07s/it]

Processing {'dataset': 'HouseTwenty', 'run': 4, 'model': 'shapelet'}


Processing:   8%|▊         | 349/4480 [3:18:11<39:05:54, 34.07s/it]


KeyboardInterrupt: 

In [ ]:
df = pl.read_parquet(f'{write_dir}/*.parquet').filter(pl.col("dataset").is_in(datasets))
df

In [ ]:
datasets

In [ ]:
gdf = df.group_by('dataset', 'model').agg([
    pl.col('test_accuracy').mean()
]).sort('test_accuracy')

In [ ]:
together_dataset = set(
    gdf.filter(pl.col('model') == 'tabpfn')['dataset']
).intersection(
    set(gdf.filter(pl.col('model') == 'quant')['dataset'])
)
v1 = gdf.filter(pl.col('model') == 'tabpfn').filter(pl.col('dataset').is_in(together_dataset)).sort('dataset')['test_accuracy']
v2 = gdf.filter(pl.col('model') == 'quant').filter(pl.col('dataset').is_in(together_dataset)).sort('dataset')['test_accuracy']
plt.scatter(v1, v2)
# plot read line y=x
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('TabPFN Test Accuracy')
plt.ylabel('QUANT Test Accuracy')
plt.title('Model Performance Comparison: TabPFN vs QUANT')
plt.grid(True)
plt.show()

In [ ]:
together_dataset = set(
    gdf.filter(pl.col('model') == 'tabpfn')['dataset']
).intersection(
    set(gdf.filter(pl.col('model') == 'raw-scale-ridge')['dataset'])
)
v1 = gdf.filter(pl.col('model') == 'tabpfn').filter(pl.col('dataset').is_in(together_dataset)).sort('dataset')['test_accuracy']
v2 = gdf.filter(pl.col('model') == 'raw-scale-ridge').filter(pl.col('dataset').is_in(together_dataset)).sort('dataset')['test_accuracy']
plt.scatter(v1, v2)
# plot read line y=x
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('TabPFN Test Accuracy')
plt.ylabel('Raw-scale Ridge Classifier Accuracy')
plt.title('Model Performance Comparison: TabPFN vs Raw-scale Ridge Classifier')
plt.grid(True)
plt.show()

In [ ]:
v1 = gdf.filter(pl.col('model') == 'raw-scale-ridge').sort('dataset')['test_accuracy']
v2 = gdf.filter(pl.col('model') == 'quant').sort('dataset')['test_accuracy']
plt.scatter(v1, v2)
# plot read line y=x
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('Raw-scale Ridge Classifier Accuracy')
plt.ylabel('QUANT Classifier Accuracy')
plt.title('Model Performance Comparison')
plt.grid(True)
plt.show()

In [ ]:
v1 = gdf.filter(pl.col('model') == 'quant').sort('dataset')['test_accuracy']
v2 = gdf.filter(pl.col('model') == 'minirocket').sort('dataset')['test_accuracy']
plt.scatter(v1, v2)
# plot read line y=x
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('QUANT Classifier Accuracy')
plt.ylabel('MiniRocket Classifier Accuracy')
plt.title('Model Performance Comparison')
plt.grid(True)
plt.show()

In [ ]:
v1 = gdf.filter(pl.col('model') == 'catch22').sort('dataset')['test_accuracy']
v2 = gdf.filter(pl.col('model') == 'minirocket').sort('dataset')['test_accuracy']
plt.scatter(v1, v2)
# plot read line y=x
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('Catch22 Classifier Accuracy')
plt.ylabel('MiniRocket Classifier Accuracy')
plt.title('Model Performance Comparison')
plt.grid(True)
plt.show()

In [ ]:
v1 = gdf.filter(pl.col('model') == 'catch22').sort('dataset')['test_accuracy']
v2 = gdf.filter(pl.col('model') == 'raw-scale-ridge').sort('dataset')['test_accuracy']
plt.scatter(v1, v2)
# plot read line y=x
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('Catch22 Classifier Accuracy')
plt.ylabel('Raw-scale Ridge Classifier Accuracy')
plt.title('Model Performance Comparison')
plt.grid(True)
plt.show()

In [ ]:
gdf = df.group_by('dataset', 'model').agg([
    pl.col('training_time').mean()
]).sort('dataset', 'model')

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 20))
sns.stripplot(data=gdf, y="dataset", x="training_time", hue="model", dodge=False)
plt.grid(True)
plt.xscale('log')
plt.tight_layout()
plt.show()

In [ ]:
gdf = df.group_by('dataset', 'model').agg([
    pl.col('test_accuracy').mean()
]).sort('dataset', 'model')

In [ ]:
df['dataset'].unique()

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 20))
sns.stripplot(data=gdf, y="dataset", x="test_accuracy", hue="model", dodge=False)
plt.grid(True)
plt.xscale('log')
plt.tight_layout()
plt.show()

In [ ]:
gdf = df.group_by('dataset', 'model').agg([
    pl.col('test_accuracy').mean(),
    pl.col('training_time').mean()
]).sort('dataset', 'model')

In [ ]:
sns.scatterplot(data=gdf, x='training_time', y='test_accuracy', hue='model')
plt.xscale('log')